### Machine Learning model 
ARIMA Model

### COntributions
Anuraag Reddy <br>
Sujaya Dahagam

Fits an ARIMA model with predefined parameters to forecast average property prices.<br>
Calculates the Root Mean Square Error (RMSE) to assess model accuracy.<br>
DOnw below is the output that shows the RMSE and the last five predicted values from the model.

In [30]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
with warnings.catch_warnings():
    warnings.simplefilter("ignore",ValueWarning)

# Load the dataset
file_path = 'C:/Users/udayv/Downloads/zillow_dataset_2023.csv' 
df = pd.read_csv(file_path)

# Filter data for Chicago
chicago_data = df[df['City'] == 'Chicago']

# Assume we are interested in average price across all zip codes in Chicago
chicago_prices = chicago_data.iloc[:, 9:].mean()

# Converting to a time series
chicago_ts = pd.Series(chicago_prices)
chicago_ts.index = pd.to_datetime(chicago_ts.index)
chicago_ts = chicago_ts.asfreq('M')

# Splitting the data into train and test sets
train_size = int(len(chicago_ts) * 0.8)
train, test = chicago_ts[0:train_size], chicago_ts[train_size:]

# Fit an ARIMA model (order (p,d,q) should be chosen based on data analysis)
model = ARIMA(train, order=(5,1,0))
model_fit = model.fit()

# Forecast
forecast = model_fit.forecast(steps=len(test))

# Calculate RMSE for accuracy test
rmse = sqrt(mean_squared_error(test, forecast))

# Print RMSE and the last 5 predicted values
print(f'RMSE: {rmse}')
print(f'Last 5 Predicted Prices:\n{forecast[-5:]}')


RMSE: 19526.479958104628
Last 5 Predicted Prices:
2023-05-31    264296.655839
2023-06-30    264296.655839
2023-07-31    264296.655839
2023-08-31    264296.655839
2023-09-30    264296.655839
Freq: M, Name: predicted_mean, dtype: float64


In [31]:
#chicago_ts = chicago_ts.asfreq('M')  # Set frequency to monthly
def calculate_mape(actual, forecast):
    """Calculate Mean Absolute Percentage Error."""
    return np.mean(np.abs((actual - forecast) / actual)) * 100

# Load the dataset
file_path = 'C:/Users/udayv/Downloads/zillow_dataset_2023.csv'  
df = pd.read_csv(file_path)

# Filter data for Chicago
chicago_data = df[df['City'] == 'Chicago']
chicago_prices = chicago_data.iloc[:, 9:].mean()
chicago_ts = pd.Series(chicago_prices)
chicago_ts.index = pd.to_datetime(chicago_ts.index)
chicago_ts = chicago_ts.asfreq('M')

# Splitting the data into train and test sets
train_size = int(len(chicago_ts) * 0.8)
train, test = chicago_ts[0:train_size], chicago_ts[train_size:]

# Fit an ARIMA model
model = ARIMA(train, order=(5,1,0))
model_fit = model.fit()

# Forecast
forecast = model_fit.forecast(steps=len(test))

# Calculate RMSE and MAPE
rmse = sqrt(mean_squared_error(test, forecast))
mape = calculate_mape(test, forecast)

# Print RMSE and MAPE
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')


RMSE: 19526.479958104628
MAPE: 5.493129796387295%


We will try improving the model accuracy using SARIMAX 
We use auto arima to find the best ARIMA model parameters
You can see the actual vs the predicted values in the output section

In [32]:
# Load your dataset
#file_path = 'C:/Users/udayv/Downloads/zillow_dataset_2023.csv'

best_model = auto_arima(train, start_p=1, start_q=1,
                        max_p=5, max_q=5, m=12,
                        start_P=0, seasonal=True,
                        d=None, D=1, trace=True,
                        error_action='ignore',  
                        suppress_warnings=True, 
                        stepwise=True)

print(best_model.summary())

# Fit the best SARIMAX model (which includes ARIMA)
model = SARIMAX(train, 
                order=best_model.order, 
                seasonal_order=best_model.seasonal_order)
model_fit = model.fit()

# Forecast
forecast = model_fit.forecast(steps=len(test))

# Combine actual and forecasted values into a DataFrame for comparison
improved_comparison_df = pd.DataFrame({'Actual Value': test, 'Predicted Value': forecast})

# Calculate RMSE and MAPE for the improved model
rmse_improved = sqrt(mean_squared_error(test, forecast))
mape_improved = np.mean(np.abs((test - forecast) / test)) * 100

# Print the improved metrics
print(f'Improved RMSE: {rmse_improved}')
print(f'Improved MAPE: {mape_improved}%')

# Print the first few rows of the comparison DataFrame
print(improved_comparison_df.head())


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=3757.825, Time=2.18 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=3773.541, Time=0.06 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=3775.407, Time=0.94 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=3776.259, Time=0.44 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=3756.831, Time=0.47 sec
 ARIMA(1,1,1)(1,1,0)[12]             : AIC=3757.869, Time=1.73 sec
 ARIMA(1,1,1)(1,1,1)[12]             : AIC=3757.186, Time=2.65 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=3774.286, Time=0.10 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=3773.427, Time=0.24 sec
 ARIMA(2,1,1)(0,1,0)[12]             : AIC=3758.712, Time=0.89 sec
 ARIMA(1,1,2)(0,1,0)[12]             : AIC=3758.711, Time=0.81 sec
 ARIMA(0,1,2)(0,1,0)[12]             : AIC=3776.963, Time=0.15 sec
 ARIMA(2,1,0)(0,1,0)[12]             : AIC=3773.140, Time=0.44 sec
 ARIMA(2,1,2)(0,1,0)[12]             : AIC=inf, Time=1.16 sec
 ARIMA(1,1,1)(0,1,0)[12]

AS you can see above we have trained the model and our further work will be to focus on accuarcy of the mdoel part and also try
for diffrent varibales id the time permits.


Link to the github repository -> https://github.com/VooraUdayBhaskar/Real-Estate_Price_Prediction